<a href="https://colab.research.google.com/github/thainazanfolin/min_sleep_efficiency/blob/main/Projeto_Min_Sleep_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

todo: ESCREVER INFORMAÇÃO INICIAIS


Vamos importar a nossa base de dados. Optamos por trazer por URL para facilitar o trabalho de todos os membros da equipe em um só colab. <br> Porém, também pode ser feito subindo a base aqui no Colab, normalmente. <br>
Vamos começar!

In [4]:
import pandas as pd

url = 'https://raw.githubusercontent.com/thainazanfolin/min_sleep_efficiency/refs/heads/main/Sleep_Efficiency_table.csv'
df = pd.read_csv(url, delimiter=';')

# Verifique as primeiras linhas do DataFrame
print(df.head(100))


     ID  Age  Gender           Bedtime       Wakeup time  Sleep duration  \
0     1   65  Female  06/03/2021 01:00  06/03/2021 07:00             6.0   
1     2   69    Male  05/12/2021 02:00  05/12/2021 09:00             7.0   
2     3   40  Female  25/05/2021 21:30  25/05/2021 05:30             8.0   
3     4   40  Female  03/11/2021 02:30  03/11/2021 08:30             6.0   
4     5   57    Male  13/03/2021 01:00  13/03/2021 09:00             8.0   
..  ...  ...     ...               ...               ...             ...   
95   96   61  Female  10/07/2021 02:00  10/07/2021 09:00             7.0   
96   97   41    Male  15/02/2021 21:00  15/02/2021 04:00             7.0   
97   98   53    Male  02/11/2021 23:00  02/11/2021 06:30             7.5   
98   99   32  Female  12/01/2021 02:00  12/01/2021 11:00             9.0   
99  100   65    Male  07/12/2021 01:30  07/12/2021 09:30             8.0   

    Sleep efficiency  REM sleep percentage  Deep sleep percentage  \
0               0.